In [1]:
import os
import gc
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = str(3)

In [2]:
import cudf
import pandas as pd
import xgboost as xgb
from sklearn import datasets
cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)
pd.set_option('display.max_columns', 500)

In [4]:
features = [
            'user_user_count',
            'user_item_count',
            'remember_ratio',
            'price',
            'price_power_0_mean',
            'price_power_1_mean',
            'price_power_2_mean',
            'price_power_3_mean',
            'price_power_5_mean',
            'nn_logits',
            'es',
            'fr',
            'it',
           ]
bad = [4,7,#1
       11,14,#2
       18,21,#3
       25,28,#4
       32,35,#5
       39,42,#6
       44,45,46,47,48,49,#7
       60,62,63,#9
       66,67,70,#10
       72,73,74,75,76,77,#11
       80,81,84,#12
       87,88,89,90,91,#13
       101,102,103,104,105,#15
       108,109,110,111,112,#16
       115,116,117,118,119,#17
       123,126,#18
       130,133,#19
       144,147,#21
       151,154,#22
       158,151,#23
       203,206,
       ]#DE
bad = [
       72,73,74,75,76,77,#11 string
       101,102,103,104,105,#15 brand
       108,109,110,111,112,#16 author
       115,116,117,118,119,#17 material
       182,189,196,30*7,
       ]#DE
feature_list =  [[200,206]]
for i in [1,2,3,4,5,6,7,9,10,11,12,13,15,16,17,18,19,21,23,24,25,26,27,28,31]:
    feature_list.append([(i-1)*7+1,i*7])
for i in feature_list:
    for j in range(i[0],i[1]+1):
        if j in bad:
            continue
        features+=['ui_feat'+str(j)]

In [5]:
valid = []
data_set = ['FR']
for locale__2 in data_set :#['ES','FR','IT']
    temp_valid = pd.read_parquet('valid5_data_'+locale__2+'_0~0.pqt')
    a = [0]*len(temp_valid)
    b = [0]*len(temp_valid)
    c = [0]*len(temp_valid)
    if locale__2 == 'ES':
        a = [1]*len(temp_valid)
    if locale__2 == 'FR':
        b = [1]*len(temp_valid)
    if locale__2 == 'IT':
        c = [1]*len(temp_valid)
    temp_valid['es']=a
    temp_valid['fr']=b
    temp_valid['it']=c
    valid.append(temp_valid)
    
valid = pd.concat(valid).reset_index(drop=True)
for row in tqdm(feature_list):
    temp_df = []
    for locale__2 in data_set :#['ES','FR','IT']
        temp_df.append(pd.read_parquet('valid5_data_'+locale__2+'_'+str(row[0])+'~'+str(row[1])+'.pqt'))
    temp_df = pd.concat(temp_df).reset_index(drop=True)
    
    for j in range(row[0],row[1]+1):
        if j in bad:
            continue
        valid['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
    del temp_df
X_valid = valid[features]
y_valid = valid['target']
dvalid = xgb.DMatrix(cudf.DataFrame(X_valid),cudf.DataFrame( y_valid), group=[250] * (len(X_valid)//250) ) 

 42%|████▏     | 11/26 [00:00<00:00, 53.54it/s]/tmp/ipykernel_1996899/3509222865.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  valid['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996899/3509222865.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  valid['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996899/3509222865.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

# GPU

In [6]:
class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, batch_size=1024*256):
        self.features = features
        self.target = 'target'
        self.df = df
        self.it = 0 # set iterator to 0
        self.batch_size = batch_size
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        if self.it == self.batches:
            return 0 # Return 0 when there's no more batch.
        
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        dt = cudf.DataFrame(self.df.iloc[a:b])
        input_data(data=dt[self.features], label=dt[self.target]) #, weight=dt['weight'])
        self.it += 1
        return 1

# mrr100

In [7]:
import numpy as np
from typing import Tuple
def mrr_at_k(predt: np.ndarray, dtrain: xgb.DMatrix, k: int) -> Tuple[str, float]:
    y = dtrain.get_label()
    group_sizes = dtrain.get_group()
    assert len(y) == sum(group_sizes), "group sizes must match data length"
    assert len(predt) == sum(group_sizes), "group sizes must match prediction length"
    mrr_sum = 0.
    mrr_count = 0
    start_idx = 0
    for i, group_size in enumerate(group_sizes):
        end_idx = start_idx + group_size
        group_y = y[start_idx:end_idx]
        group_predt = predt[start_idx:end_idx]
        order = np.argsort(-group_predt)
        ranks = np.zeros_like(order)
        ranks[order] = np.arange(len(group_predt))
        reciprocal_ranks = 1. / (ranks + 1.)
        if (group_y > 0).sum() == 0:
            mrr = 0.
        else:
            mrr = np.mean(reciprocal_ranks[group_y > 0])
        mrr_sum += mrr
        mrr_count += 1
        start_idx = end_idx
    mrr_at_k = mrr_sum / mrr_count if mrr_count > 0 else 0.
    return f"mrr@{k}", mrr_at_k

# train

In [8]:
train = []
data_set = ['ES','FR','IT']
for locale__2 in data_set :#['DE','JP','UK']
    temp_train = pd.read_parquet('train5_data_'+locale__2+'_0~0.pqt')
    a = [0]*len(temp_train)
    b = [0]*len(temp_train)
    c = [0]*len(temp_train)
    if locale__2 == 'ES':
        a = [1]*len(temp_train)
    if locale__2 == 'FR':
        b = [1]*len(temp_train)
    if locale__2 == 'IT':
        c = [1]*len(temp_train)
    temp_train['es']=a
    temp_train['fr']=b
    temp_train['it']=c
    train.append(temp_train)
    
train = pd.concat(train).reset_index(drop=True)
for row in tqdm(feature_list):
    temp_df = []
    for locale__2 in data_set :#['DE','JP','UK']
        temp_df.append(pd.read_parquet('train5_data_'+locale__2+'_'+str(row[0])+'~'+str(row[1])+'.pqt'))
    temp_df = pd.concat(temp_df).reset_index(drop=True)
    
    for j in range(row[0],row[1]+1):
        if j in bad:
            continue
        train['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
    del temp_df

Xy_train = IterLoadForDMatrix(train, features)
dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=256)
dtrain.set_group([250] * (len(train)//250))

 62%|██████▏   | 16/26 [02:36<02:12, 13.28s/it]/tmp/ipykernel_1996899/4049417552.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996899/4049417552.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['ui_feat'+str(j)] = temp_df['ui_feat'+str(j)]
/tmp/ipykernel_1996899/4049417552.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

In [9]:
target = valid['target'].tolist()
rank = valid['rank'].tolist()
mrr = 0
for i in range(len(rank)//250):
    for j in range(100):
        if target[i*250+j] == 1:
            mrr+=1/(1+j)
mrr/len(rank)*250

0.401391432584136

In [ ]:

xgb_parms = { 
        'max_depth':4, 
        'learning_rate':0.07, 
        'subsample':0.5,
        'colsample_bytree':0.6,
        # 'colsample_bylevel': 0.6,
        # 'min_child_weight': 5,
        # 'gamma':0.1,
        'reg_lambda':0.05,
        'eval_metric':'map',
        'objective':'binary:logistic',
        # 'objective':'rank:pairwise',
        'scale_pos_weight':8,

        'tree_method':'gpu_hist',
        'predictor':'gpu_predictor',
        'num_feature':len(features),
        'random_state':42
    }
model = None
_ = xgb.train(xgb_parms, 
    dtrain=dtrain,
    evals=[(dtrain,'train'),(dvalid,'valid')],
    feval=lambda predt, dtrain: mrr_at_k(predt, dtrain, k=100),
    num_boost_round=1,
    early_stopping_rounds=0,
    verbose_eval=1,
    xgb_model=model)
model = xgb.train(xgb_parms, 
    dtrain=dtrain,
    evals=[(dtrain,'train'),(dvalid,'valid')],
    num_boost_round=3500,
    early_stopping_rounds=300,
    verbose_eval=100,
    xgb_model=model)
_ = xgb.train(xgb_parms, 
    dtrain=dtrain,
    evals=[(dtrain,'train'),(dvalid,'valid')],
    feval=lambda predt, dtrain: mrr_at_k(predt, dtrain, k=100),
    num_boost_round=1,
    early_stopping_rounds=0,
    verbose_eval=1,
    xgb_model=model)

model.save_model(f'XGB_locale_sum4.xgb')
#[0]	train-map:0.61478	train-mrr@100:0.45069	valid-map:0.60668	valid-mrr@100:0.46838

In [15]:
_ = xgb.train(  xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                feval=lambda predt, dtrain: mrr_at_k(predt, dtrain, k=100),
                num_boost_round=1,
                early_stopping_rounds=0,
                verbose_eval=1,
                xgb_model=model)#0.36880

[0]	train-map:0.61479	train-mrr@100:0.45071	valid-map:0.60578	valid-mrr@100:0.46748


In [ ]:

model.save_model(f'XGB_locale_sum4.xgb')

### features importance

In [ ]:
a = features#list(model.get_score(importance_type='gain').keys())
b = list(model.get_score(importance_type='gain').values())
Z = zip(b, a)  
Z = sorted(Z, reverse=True)
b, a = zip(*Z)
for i,j in zip(b, a):
    print(i,'\t',j)

In [ ]:
import os
 
pid = os.getpid()
!kill -9 $pid

# test

In [24]:
# # Test
xgb_parms = { 
            'max_depth':4, 
            'learning_rate':0.07, 
            'subsample':0.5,
            'colsample_bytree':0.6,
            # 'gamma':0.1,
            'reg_lambda':0.05,
            'eval_metric':'map',
            'objective':'binary:logistic',
            # 'objective':'rank:pairwise',
            'scale_pos_weight':8,
            'tree_method':'gpu_hist',
            'predictor':'gpu_predictor',
            'random_state':42
        }
model = xgb.Booster()
# model.set_param(xgb_parms)


In [16]:
recall1 = []
le = 0

for locale__ in tqdm(['ES','FR','IT']):
    model = xgb.Booster()
    model.load_model(f'XGB_locale_sum4.xgb')
    data = cudf.read_parquet('test5_data_'+locale__+'_0~0.pqt')
    a = [0]*len(data)
    b = [0]*len(data)
    c = [0]*len(data)
    if locale__ == 'ES':
        a = [1]*len(data)
    if locale__ == 'FR':
        b = [1]*len(data)
    if locale__ == 'IT':
        c = [1]*len(data)
    data['es']=a
    data['fr']=b
    data['it']=c
    if len(data)==0:
        break
    for row in feature_list:
        temp_df = cudf.read_parquet('test5_data_'+locale__+'_'+str(row[0])+'~'+str(row[1])+'.pqt')
        for j in range(row[0],row[1]+1):
            if j in bad:
                continue
            data['ui_feat'+str(j)] = cudf.DataFrame(temp_df['ui_feat'+str(j)])
    le+=len(data)

    dtest = xgb.DMatrix(data=cudf.DataFrame(data[features]))
    preds = model.predict(dtest)
    preds = np.array(preds)*np.log(np.e+5*data['sum_pop_score'].to_pandas())
    del dtest
    predictions = cudf.DataFrame(data[['user', 'item']])
    predictions['pred'] = preds
    predictions = predictions.sort_values(['user', 'pred'], ascending=[True, False]).reset_index(drop=True)
    predictions['rk'] = predictions.groupby('user').item.cumcount().astype('int16')
    predictions = predictions.loc[predictions.rk < 150]
    predictions = predictions.to_pandas()
    predictions = predictions.groupby('user').item.apply(list).to_frame().reset_index()
    predictions.columns = ['test_user', 'next_item_prediction']
    recall1 += predictions['next_item_prediction'].tolist()
    del predictions

100%|██████████| 3/3 [00:40<00:00, 13.35s/it]


### outout result

In [17]:
import pickle
product_id2asin = pickle.load(open('../data/product_id2asin.pkl', 'rb'))
locale = pd.read_pickle('../data/test_data2_p2.dataset').reset_index(drop=True)['locale']
result = [[product_id2asin[x] for x in row[:100]] for row in recall1]
result = pd.DataFrame({'locale':locale,'next_item_prediction':result})
result.to_parquet( 'submission_locale_sum.parquet', engine='pyarrow')
result

,locale,next_item_prediction
0,ES,"[B074581R8Y, B074RZKWMD, B0745795KF, B07GTWD7Z..."
1,ES,"[B09J4T4JF5, B09NT33LZN, B09M8LNB61, B09NQKCQW..."
2,ES,"[B0B461V1RB, B0B4643ZJN, B0B461KYQY, B0B38J377..."
3,ES,"[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8..."
4,ES,"[B09G97SLGS, B09G9DMQ7M, B09G99D95Q, B09G9LF91..."
...,...,...
34685,IT,"[B01F545TZQ, B01BBW9OAC, B09QH4W4LS, B01BBW9V3..."
34686,IT,"[B08DYGN5W7, B09TW215N7, B08DJ18C85, B00IBDVTS..."
34687,IT,"[B07ZH7HYDX, B07KXXCDS2, B00P2J6IP0, B08MNSMFP..."
34688,IT,"[B09RBZ3XDV, B09T5N86FW, B09V52D3SQ, B0B245K5X..."


In [46]:
import pickle
product_id2asin = pickle.load(open('../data/product_id2asin.pkl', 'rb'))
locale = pd.read_pickle('../data/test_data2_p2.dataset').reset_index(drop=True)['locale']
result = [[product_id2asin[x] for x in row[:100]] for row in recall1]
result = pd.DataFrame({'locale':locale,'next_item_prediction':result})
result.to_parquet( 'submission_locale_sum.parquet', engine='pyarrow')
result

,locale,next_item_prediction
0,ES,"[B074581R8Y, B074RZKWMD, B0745795KF, B07457KX2..."
1,ES,"[B09J4T4JF5, B09NT33LZN, B09M8LNB61, B09NQKCQW..."
2,ES,"[B0B461V1RB, B0B4643ZJN, B0B461KYQY, B0B38J377..."
3,ES,"[B07R4VG9X3, B07R4WKSSV, B07QZ35DY6, B07QXZJK8..."
4,ES,"[B09G97SLGS, B09G9DMQ7M, B09G99D95Q, B09G9LF91..."
...,...,...
34685,IT,"[B01F545TZQ, B01BBW9OAC, B09QH4W4LS, B01BBW9V3..."
34686,IT,"[B08DYGN5W7, B09TW215N7, B08DJ18C85, B00IBDVTS..."
34687,IT,"[B07ZH7HYDX, B07KXXCDS2, B00P2J6IP0, B08MNSMFP..."
34688,IT,"[B09T5N86FW, B09RBZ3XDV, B09V52D3SQ, B0B245K5X..."
